##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Title

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/abhichou4/addons/blob/tutorialAvg/docs/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/abhichou4/addons/blob/tutorialAvg/docs/tutorials/average_optimizers_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This notebook demonstrates how to use Moving Average Optimizer along with the Model Average Checkpoint from tensorflow addons pagkage.




## Moving Averaging 

> The advantage of Moving Averaging is that they are less prone to rampant loss shifts or irregular data representation in the latest batch. It gives a smooothened and a more genral idea of the model training until some point.

## Stocastic Averaging

> Stocastic Weight Averaging converges to wider optimas. By doing so, it resembles geometric ensembeling. SWA is a simple method to improve model performance when used as a wrapper around other optimizers and averaging results from different points of trajectory of the inner optimizer.

## Model Average Checkpoint 

> ```callbacks.ModelCheckpoint``` doesn't give you the option to save moving average weights in the middle of traning, which is why Model Average Optimizers required a custom callback. Using the ```update_weights``` parameter, ```ModelAverageCheckpoint``` allows you to:
1.   Assign the moving average weights to the model, and save them.
2.   Keep the old non-averaged weights, but the saved model uses the average weights.

## Setup

In [2]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf

TensorFlow 2.x selected.


In [3]:
!pip install --upgrade tfa-nightly

import tensorflow_addons as tfa

Requirement already up-to-date: tfa-nightly in /usr/local/lib/python3.6/dist-packages (0.9.0.dev20200225)


In [0]:
import tensorflow_datasets as tfds
import numpy as np
from matplotlib import pyplot as plt
import os

## Build Model 

In [0]:
def create_model(opt):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),                         
    tf.keras.layers.Dense(64, activation='relu', name='dense_1'),
    tf.keras.layers.Dense(64, activation='relu', name='dense_2'),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=opt,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

## Prepare Dataset

In [0]:
#Load Fashion MNIST dataset
train, test = tf.keras.datasets.fashion_mnist.load_data()

images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

test_images, test_labels = test

We will be comparing three optimizers here:

*   Unwrapped SGD
*   SGD with Moving Average
*   SGD with Stochastic Weight Averaging

And see how they perform with the same model.

In [0]:
#Optimizers 
sgd = tf.keras.optimizers.SGD(0.01)
moving_avg_sgd = tfa.optimizers.MovingAverage(sgd)
stocastic_avg_sgd = tfa.optimizers.SWA(sgd)

Both ```MovingAverage``` and ```StocasticAverage``` optimers use ```ModelAverageCheckpoint```.

In [0]:
#Callback 
checkpoint_path = "./training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)
avg_callback = tfa.callbacks.average_model_checkpoint.AverageModelCheckpoint(filepath=checkpoint_dir, 
                                                                            update_weights=True)

## Train Model


### Vanilla SGD Optimizer 

In [9]:
#Build Model
model = create_model(sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Train for 1875 steps
Epoch 1/5
1874/1875 [============================>.] - ETA: 0s - loss: 0.7737 - accuracy: 0.7426
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.7737 - accuracy: 0.7426
Epoch 2/5
1869/1875 [============================>.] - ETA: 0s - loss: 0.5041 - accuracy: 0.8245
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5037 - accuracy: 0.8247
Epoch 3/5
1866/1875 [============================>.] - ETA: 0s - loss: 0.4575 - accuracy: 0.8408
Epoch 00003: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4576 - accuracy: 0.8407
Epoch 4/5
1855/1875 [============================>.] - ETA: 0s - loss: 0.4298 - accuracy: 0.8483
Epoch 00004: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4299 - accuracy: 0.8483
Epoch 5/5
1860/1875 [============================>.] - ETA: 0s - lo

In [10]:
#Evalute results
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 93.3245 - accuracy: 0.7870
Loss : 93.32448555297852
Accuracy : 0.787


### Moving Average SGD

In [11]:
#Build Model
model = create_model(moving_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Train for 1875 steps
Epoch 1/5
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
1857/1875 [============================>.] - ETA: 0s - loss: 0.7890 - accuracy: 0.7370
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.7866 - accuracy: 0.7377
Epoch 2/5
1871/1875 [============================>.] - ETA: 0s - loss: 0.5071 - accuracy: 0.8219
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5071 - accuracy: 0.8219
Epoch 3/5
1865/1875 [============================>.] - ETA: 0s - loss: 0.4584 - accuracy: 0.8380
Epoch 00003: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4583 - accuracy: 0.8381
Epoch 4/5
1867/1875 [============================>.] - ETA: 0s - loss: 0.4309 - accuracy: 0.8487
Epoch 00004: saving model to ./training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4304 - acc

In [12]:
#Evalute results
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 106.2339 - accuracy: 0.7722
Loss : 106.23389839782715
Accuracy : 0.7722


### Stocastic Weight Average SGD 

In [13]:
#Build Model
model = create_model(stocastic_avg_sgd)

#Train the network
model.fit(fmnist_train_ds, epochs=5, callbacks=[cp_callback])

Train for 1875 steps
Epoch 1/5
1872/1875 [============================>.] - ETA: 0s - loss: 0.7673 - accuracy: 0.7464
Epoch 00001: saving model to ./training
1875/1875 [==============================] - 6s 3ms/step - loss: 0.7670 - accuracy: 0.7465
Epoch 2/5
1871/1875 [============================>.] - ETA: 0s - loss: 0.5034 - accuracy: 0.8248
Epoch 00002: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5034 - accuracy: 0.8247
Epoch 3/5
1858/1875 [============================>.] - ETA: 0s - loss: 0.4557 - accuracy: 0.8407
Epoch 00003: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4561 - accuracy: 0.8407
Epoch 4/5
1860/1875 [============================>.] - ETA: 0s - loss: 0.4266 - accuracy: 0.8509
Epoch 00004: saving model to ./training
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4273 - accuracy: 0.8507
Epoch 5/5
1859/1875 [============================>.] - ETA: 0s - lo

In [14]:
#Evalute results
loss, accuracy = model.evaluate(test_images, test_labels, batch_size=32, verbose=2)
print("Loss :", loss)
print("Accuracy :", accuracy)

10000/10000 - 0s - loss: 85.0133 - accuracy: 0.7989
Loss : 85.01332214202881
Accuracy : 0.7989
